# Minesweeper game

## Rules:

Minesweeper has very simple rules. The playing field is divided into cells, some of which are mined. To win, you need to open all the cells without hitting the mines. Digits are displayed in open cells, each digit is the number of mines in neighboring cells. With this information, you can determine which cells contain mines.

The game has 3 difficulty levels: easy, medium and hard. They are differ by the amount of mines placed on the field and by the field size.

In [1]:
import random
import numpy as np

### Adding mines to the playing field

In [2]:
def get_mines_position(m: int, n: int, number_mines: int):
    '''
    the function randomy select place for k mines on the playing field
    '''
    mines_pos = []
    while len(mines_pos) < number_mines:
        for mine in range(number_mines):
            mine_pos_x = random.randint(0, m - 1)
            mine_pos_y = random.randint(0, n - 1)
            if [mine_pos_x, mine_pos_y] in mines_pos:
                continue
            else:
                mines_pos.append([mine_pos_x, mine_pos_y])
    return mines_pos


def place_mines(mines_positions: list, playing_field):
    '''
    the functions insertes mines to the playing field
    according to the coordinates recieved from get_mines_position()
    '''
    for mine in mines_positions:
        playing_field[mine[0]][mine[1]] = "*"
    return playing_field


def is_mine(field, i: int, j: int):
    '''
    is a helping function to determine weather there is a mine in the neighbouring cells
    '''
    n, m = field.shape
    if (0 <= i < n) and (0 <= j < m) and field[i][j] == "*":
        return 1
    return 0


def add_mines_number(field):
    '''
    depending on the cells where mines were plased, 
    the function adds the number of mines in the neighboring cells
    and returns the playing field
    '''
    n, m = field.shape
    for i in range(n):
        for j in range(m):
            if field[i][j] == "*":
                continue
            counter = 0
            right = is_mine(field, i, j+1)
            left = is_mine(field, i, j-1)
            top = is_mine(field, i-1, j)
            bottom = is_mine(field, i+1, j)
            top_left = is_mine(field, i-1, j-1)
            top_right = is_mine(field, i-1, j+1)
            bottom_left = is_mine(field, i+1, j-1)
            bottom_right = is_mine(field, i+1, j+1)
            counter = right + left + top + bottom + top_left + top_right +\
                            bottom_left + bottom_right
            field[i][j] = counter
            
    return field

### Creating a field

In [3]:
def create_field(d_level: str):
    '''
    the function creates a playnig field m by n cells, 
    depending on the chosen difficulty level
    '''
    if d_level.lower() == 'easy':
        m = 6
        n = 4
        n_mines = 4
    elif d_level.lower() == 'medium':
        m = 8
        n = 5
        n_mines = 6
    elif d_level.lower() == 'hard':
        m = 10
        n = 6
        n_mines = 10
        
    playing_field = np.zeros((m, n), dtype=str)
    mines_rand_pos = get_mines_position(m, n, n_mines)
    playing_field_with_mines = place_mines(mines_rand_pos, playing_field)
    playing_field_with_numbers = add_mines_number(playing_field_with_mines)
    
    return playing_field_with_numbers, mines_rand_pos

### Checking user input

In [4]:
def check_user_level_input():
    '''
    checks whether the user input difficulty level correctly
    '''
    level_message = '\nPlease input the difficulty level (easy / medium / hard): '
    levels = ['easy', 'medium', 'hard']
    string = input(level_message)
    if string.lower() in levels:
        return string.lower()
    else:
        print('\nTry again! Available levels are: easy, medium, hard')
        return check_user_input()
    
    
def check_user_field_input(field_shape):
    '''
    checks whether the user input cell coordinates correctly
    '''
    input_message = '\nTo uncover cell, input cell coordinates (two digits, sep=space): '
    coordinates = input(input_message)
    coord_split = coordinates.split()
    m = int(field_shape[0])
    n = int(field_shape[1])
    if len(coord_split) == 2:
        try:
            x = int(coord_split[0]) - 1
            y = int(coord_split[1]) - 1
            if (x < 0 or x > m-1) or (y < 0 or y > n-1):
                print('\nTry again! The numbers must be within the field size!')
                print(f'Field size is {m}x{n}, indexes start from one')
                return check_user_field_input(field_shape)
            return [x, y]
        
        except ValueError:
            print('\nTry again! Input cell coordinates, example: 1 3')
            return check_user_field_input(field_shape)
    
    print('\nTry again! Must be a string of 2 digits')
    return check_user_field_input(field_shape)
    
    
def user_input_new_game():
    '''
    afrer the game finishes, ask user if he would like to play one more time
    '''
    input_message = '\nDo you want to start a new game? YES / NO: '
    answer = input(input_message)
    if answer.lower() == 'yes':
        print()
        print('*' * 20)
        print()
        return start_game()
    elif answer.lower() == 'no':
        print('\nThank you for playing!')
    else:
        print('\nThe answer must be YES or NO!')
        return user_input_new_game()

### Game progress check

In [5]:
def game_status(m, n, mines_locations, open_cells):
    '''
    Checks if the user opened all cells without mines
    '''
    game_status = True
    total_cells = m * n
    open_cells_inc_mines = len(mines_locations + open_cells)
    
    if total_cells == open_cells_inc_mines:
        print('\nCongratulations to you!!!')
        print('You have uncovered all cells without hitting the mines!')
        game_status = False
        
    return game_status


def open_zeros_cells(c_field, p_field, i, j, opened_zero_cells):
    '''
    if the player uncovers "0" cell, all the adjacent zero cells will also be uncovered
    '''
    m, n = p_field.shape
    if i < 0 or j < 0 or i >= m or j >= n or p_field[i][j] != '0' or c_field[i][j] == '0':
        return
    else:
        c_field[i][j] = '0'
        opened_zero_cells.append([i, j])
        
    open_zeros_cells(c_field, p_field, i - 1, j, opened_zero_cells)
    open_zeros_cells(c_field, p_field, i + 1, j, opened_zero_cells)
    open_zeros_cells(c_field, p_field, i, j - 1, opened_zero_cells)
    open_zeros_cells(c_field, p_field, i, j + 1, opened_zero_cells)
    return opened_zero_cells

### Initiating the start of the game

In [6]:
rules = '''Welcome to Minesweeper game! 

Rules:

The playing field is divided into cells, some of which are mined. 
To win, you need to open all the cells without hitting the mines. 
Digits are displayed in open cells, each digit is the number of mines in neighboring cells. 
With this information, you can determine which cells contain mines.

To open the cell, you need to input its coordinates.
The first digit is the row, and the second is a column of the selected cell.
Cell indexing starts from one, so if you have a field 6 by 5, you can use indexes 
from 1 to 6 for the rows and from 1 to 5 for the columns.

The game has 3 difficulty levels: easy, medium and hard. 
They are differ by the amount of mines placed on the field and by the field size.
'''

In [7]:
def start_game():
    '''
    initiates the playing process: print rules and collect 
    information for and from other functions
    '''
    game = True
    print(rules)
    level = check_user_level_input()
    playing_field, mines_indecies = create_field(level)
    m = playing_field.shape[0]
    n = playing_field.shape[1]
    covered_field = np.zeros((m, n), dtype=str)
    covered_field.fill('■')
    open_cells_record = list()

    while game:
        print()
        print(f'Field size: {m}x{n}')
        print(covered_field)
        
        cell_crd = check_user_field_input(playing_field.shape)
        row = cell_crd[0]
        col = cell_crd[1]
        
        if cell_crd in open_cells_record:
            print('\nThis cell is already open!')
            
        else:
            if playing_field[row][col] == '0':
                opened_zero_cells = open_zeros_cells(covered_field, playing_field, 
                                                row, col, list())
                open_cells_record += opened_zero_cells

            elif playing_field[row][col] == '*':
                print('\nGame over!')
                print('You have selected a cell with a mine.\n')
                game = False
                
            else: 
                open_cells_record.append(cell_crd)
                covered_field[row][col] = playing_field[row][col]
                game = game_status(m, n, mines_indecies, open_cells_record)
                
        if game is False:
            print(playing_field)
            
            
    user_input_new_game()

## Play the game

In [9]:
start_game()

Welcome to Minesweeper game! 

Rules:

The playing field is divided into cells, some of which are mined. 
To win, you need to open all the cells without hitting the mines. 
Digits are displayed in open cells, each digit is the number of mines in neighboring cells. 
With this information, you can determine which cells contain mines.

To open the cell, you need to input its coordinates.
The first digit is the row, and the second is a column of the selected cell.
Cell indexing starts from one, so if you have a field 6 by 5, you can use indexes 
from 1 to 6 for the rows and from 1 to 5 for the columns.

The game has 3 difficulty levels: easy, medium and hard. 
They are differ by the amount of mines placed on the field and by the field size.


Please input the difficulty level (easy / medium / hard): easy

Field size: 6x4
[['■' '■' '■' '■']
 ['■' '■' '■' '■']
 ['■' '■' '■' '■']
 ['■' '■' '■' '■']
 ['■' '■' '■' '■']
 ['■' '■' '■' '■']]

To uncover cell, input cell coordinates (two digits, se